In [1]:
import pandas as pd

In [2]:
resumes = pd.read_csv("D:/resume screener/datasets/resume_dataset.csv")
jobs = pd.read_csv("D:/resume screener/datasets/job_posts.csv")

In [3]:
print(resumes.shape)
print(resumes.columns)
print(resumes.head())

(962, 2)
Index(['Category', 'Resume'], dtype='object')
       Category                                             Resume
0  Data Science  Skills * Programming Languages: Python (pandas...
1  Data Science  Education Details \r\nMay 2013 to May 2017 B.E...
2  Data Science  Areas of Interest Deep Learning, Control Syste...
3  Data Science  Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4  Data Science  Education Details \r\n MCA   YMCAUST,  Faridab...


In [4]:
print(jobs.shape)
print(jobs.columns)
print(jobs.head())

(19001, 24)
Index(['jobpost', 'date', 'Title', 'Company', 'AnnouncementCode', 'Term',
       'Eligibility', 'Audience', 'StartDate', 'Duration', 'Location',
       'JobDescription', 'JobRequirment', 'RequiredQual', 'Salary',
       'ApplicationP', 'OpeningDate', 'Deadline', 'Notes', 'AboutC', 'Attach',
       'Year', 'Month', 'IT'],
      dtype='object')
                                             jobpost          date  \
0  AMERIA Investment Consulting Company\r\nJOB TI...   Jan 5, 2004   
1  International Research & Exchanges Board (IREX...   Jan 7, 2004   
2  Caucasus Environmental NGO Network (CENN)\r\nJ...   Jan 7, 2004   
3  Manoff Group\r\nJOB TITLE:  BCC Specialist\r\n...   Jan 7, 2004   
4  Yerevan Brandy Company\r\nJOB TITLE:  Software...  Jan 10, 2004   

                                               Title  \
0                            Chief Financial Officer   
1  Full-time Community Connections Intern (paid i...   
2                                Country Coordinator  

In [5]:
import re

In [6]:
def clean_text(text):
    text = re.sub(r'<.*?>', ' ', str(text))
    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)
    text = text.lower()
    text = re.sub(r'\s+', ' ', text).strip()
    return text

resumes['cleaned_text'] = resumes['Resume'].apply(clean_text)
jobs['Category'] = resumes['Category'].str.lower().str.strip()

In [7]:
jobs = jobs[['Title', 'JobDescription', 'RequiredQual']]
jobs = jobs.fillna("")
for col in ['Title', 'JobDescription', 'RequiredQual']:
    jobs[col] = jobs[col].apply(clean_text)

In [8]:
import random 
pairs = []

for idx, row in resumes.iterrows():
    resume_text = row['cleaned_text']
    category = row['Category']

    # Positive job match
    positive_jobs = jobs[jobs['Title'].str.contains(category.split()[0], case=False, na=False)]
    if not positive_jobs.empty:
        job_row = positive_jobs.sample(1).iloc[0]
        pairs.append([resume_text, job_row['JobDescription'], 1])

    # Negative job match
    negative_jobs = jobs[~jobs['Title'].str.contains(category.split()[0], case=False, na=False)]
    if not negative_jobs.empty:
        job_row = negative_jobs.sample(1).iloc[0]
        pairs.append([resume_text, job_row['JobDescription'], 0])

pairs_df = pd.DataFrame(pairs, columns=['Resume', 'Job_Desc', 'Label'])
print(pairs_df.head())


                                              Resume  \
0  skills programming languages python pandas num...   
1  skills programming languages python pandas num...   
2  education details may 2013 to may 2017 b e uit...   
3  education details may 2013 to may 2017 b e uit...   
4  areas of interest deep learning control system...   

                                            Job_Desc  Label  
0  cnfa seeks qualified candidates for the water ...      1  
1  the selected candidate will work as a team mem...      0  
2  the database administrator will be responsible...      1  
3  sunfood llc is seeking a qualified sales direc...      0  
4  working with local and global teams the role w...      1  


In [9]:
pairs_df.to_csv("D:/resume screener/datasets/training_pairs.csv", index=False)


In [10]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [11]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gg185\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\gg185\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [12]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [13]:
def clean_text(text):
    text = text.lower()
    text = text.translate(str.maketrans("","", string.punctuation))
    text = re.sub(r'/d+', '', text)
    text = text.strip()
    words = [lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words]
    return " ".join(words)

pairs_df['Resume_clean'] = pairs_df['Resume'].apply(clean_text)
pairs_df['Job_Desc_clean'] = pairs_df['Job_Desc'].apply(clean_text)

print(pairs_df[['Resume_clean', 'Job_Desc_clean', 'Label']].head())

                                        Resume_clean  \
0  skill programming language python panda numpy ...   
1  skill programming language python panda numpy ...   
2  education detail may 2013 may 2017 b e uit rgp...   
3  education detail may 2013 may 2017 b e uit rgp...   
4  area interest deep learning control system des...   

                                      Job_Desc_clean  Label  
0  cnfa seek qualified candidate water market wtm...      1  
1  selected candidate work team member company te...      0  
2  database administrator responsible managing ma...      1  
3  sunfood llc seeking qualified sale director ef...      0  
4  working local global team role initially suppo...      1  


In [14]:
from sentence_transformers import SentenceTransformer
import numpy as np

In [15]:
model_bert = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\gg185\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\gg185\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
resume_embeddings = model_bert.encode(pairs_df['Resume_clean'].tolist(), convert_to_numpy=True)
job_embeddings = model_bert.encode(pairs_df['Job_Desc_clean'].tolist(), convert_to_numpy=True)

In [17]:
X = np.hstack([resume_embeddings, job_embeddings])
y = pairs_df['Label'].values

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = LogisticRegression(max_iter=200)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.6694214876033058

Classification Report:
               precision    recall  f1-score   support

           0       0.67      0.77      0.71       196
           1       0.67      0.56      0.61       167

    accuracy                           0.67       363
   macro avg       0.67      0.66      0.66       363
weighted avg       0.67      0.67      0.67       363



In [19]:
import joblib


In [20]:
joblib.dump(clf, "resume_match_model.pkl")
joblib.dump(model_bert, "sbert_model.pkl")


['sbert_model.pkl']